In [1]:
# Mounting google drive for train images and groundtruth
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
%matplotlib inline
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.models import LdaModel
from gensim import models, corpora, similarities
import re
from nltk.stem.porter import PorterStemmer
import time
from nltk import FreqDist
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

In [0]:
import pandas as pd
train = pd.read_csv("/content/drive/My Drive/zs/data/train.csv") 
test = pd.read_csv("/content/drive/My Drive/zs/data/test.csv")

In [4]:
train.job_description[1:3].to_list() 

["En el mundo de los CRO's típicos, esta compañía está haciendo grandes olas para convertirse en una de las organizaciones clínicas más revolucionarias de terceros en el país. Son una empresa dinámica y de ritmo rápido que están creciendo en tamaño gradualmente a medida que amplían su trabajo.\r\r\r\nAhora están buscando un auditor principal para unirse a su equipo clínico calificado de GCP a tiempo completo ahora que el auditor anterior ha sido promovido internamente.\r\r\r\nLas principales responsabilidades de esta fantástica oportunidad son:\r\r\r\nPlanificar, conducir y supervisar auditorías de sistemas internos y proveedores\r\r\r\nAdministrar auditorías para clientes (auditorías previas al estudio y activas de estudio de las operaciones de la compañía)\r\r\r\nMantener la base de datos interna de QA\r\r\r\nAdministrar los procesos de mejora continua basados \u200b\u200ben observaciones de inspección de auditoría y CAPA de la compañía\r\r\r\nAlojar auditorías de acreditación ISO\r\

In [51]:
train[1:3]

,job_no,job_description,job_type,category,clean_text
1,Id-22925,"En el mundo de los CRO's típicos, esta compañí...",Permanent,Clinical Research,"en el mundo de los cro's t picos, esta compa ..."
2,Id-1321,Asha Mistry of Umbilical Life is recruiting an...,Contract/Interim,Pharmaceutical Marketing,asha mistry of umbilical life is recruiting an...


In [55]:
train.clean_text[100:113].to_list() 

["role  cell therapy account manager   oncology   haematology   \npackage  negotiable basic salary, car, pension scheme, private health, bonus    \nlocation  midlands covering west midlands, east midlands, cambridgeshire   \nexciting opportunity for a hospital specialist, key account specialist or medical science liaison with a proven track record of success to introduce innovative oncology therapies in the uk   \nthe client is a biopharmaceutical company engaged in the development of innovative cancer immunotherapies with a goal of providing rapid, long term durable response and eliminating the burden of chronic care  the company is focused on chimeric antigen receptor (car) and t cell receptor (tcr) engineered cell therapies designed to empower the immune system's ability to recognise and kill tumours  with headquarters in stockley park, uk they currently adding additional talent as it focuses on developing the european region in support of providing maximum access to patients with u

In [54]:
train[100:113]

,job_no,job_description,job_type,category,clean_text
100,Id-9507,Role: Cell Therapy Account Manager - Oncology ...,Permanent,Manufacturing & Operations,role cell therapy account manager oncology ...
101,Id-23851,Rédacteur médical avancé - basé au Royaume-Uni...,Permanent,Clinical Research,r dacteur m dical avanc bas au royaume uni...
102,Id-27123,Clinical Research Associate- Madrid\r\r\r\nDie...,Permanent,Science,clinical research associate madrid \ndie ma...
103,Id-18850,Jocelyn at CK QLS is currently recruiting for...,Contract/Temp,Science,jocelyn at ck qls is currently recruiting for...
104,Id-2936,I am partnered with a fast-growing global bout...,Permanent,Science,i am partnered with a fast growing global bout...
105,Id-28322,Jim Gleeson rekrutiert für einen Quality Assur...,Contract/Temp,Quality-assurance,jim gleeson rekrutiert f r einen quality assur...
106,Id-8531,Job Title: Senior Project Manager\r\r\r\nLocat...,Permanent,Science,job title senior project manager \nlocation...
107,Id-13601,Quintiles and IMS Health have come together to...,Permanent,"Pharmaceutical, Healthcare and Medical Sales",quintiles and ims health have come together to...
108,Id-3059,After study start and hand-over from the DSM ...,Contract/Interim,Clinical Research,after study start and hand over from the dsm ...
109,Id-2674,"As an integral part of everything we do, our e...",Permanent,Science,"as an integral part of everything we do, our e..."


In [0]:
import nltk
nltk.download("popular")

In [0]:
def initial_clean(text):
    """
    Function to clean text of websites, email addresess and any punctuation
    We also lower case the text
    """
    text = re.sub("((\S+)?(http(s)?)(\S+))|((\S+)?(www)(\S+))|((\S+)?(\@)(\S+)?)", " ", text)
    text = re.sub("[^a-zA-Z ]", "", text)
    text = text.lower() # lower case the text
    text = nltk.word_tokenize(text)
    return text

stop_words = stopwords.words('english')
def remove_stop_words(text):
    """
    Function that removes all stopwords from text
    """
    return [word for word in text if word not in stop_words]

stemmer = PorterStemmer()
def stem_words(text):
    """
    Function to stem words, so plural and singular are treated the same
    """
    try:
        text = [stemmer.stem(word) for word in text]
        text = [word for word in text if len(word) > 1] # make sure we have no 1 letter words
    except IndexError: # the word "oed" broke this, so needed try except
        pass
    return text

def apply_all(text):
    """
    This function applies all the functions above into one
    """
    return stem_words(remove_stop_words(initial_clean(text))) 

In [0]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

In [0]:
train["clean_text"]=train.job_description
train = standardize_text(train, "clean_text")

In [0]:
X = train[['job_no','job_description','clean_text']]
y = train['category']


In [27]:
X.head()

,job_no,job_description,clean_text
0,Id-12765,Zest Scientific is searching for an accomplis...,zest scientific is searching for an accomplis...
1,Id-22925,"En el mundo de los CRO's típicos, esta compañí...","en el mundo de los cro's t picos, esta compa ..."
2,Id-1321,Asha Mistry of Umbilical Life is recruiting an...,asha mistry of umbilical life is recruiting an...
3,Id-9473,Sales Representative - Laser in der Medizinte...,sales representative laser in der medizinte...
4,Id-14952,Field Service Engineer - Life Science\r\r\r\nC...,field service engineer life science \ncamb...


In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train.clean_text, y, test_size=0.3, random_state=42)

In [0]:
from sklearn.pipeline import Pipeline
# from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score
# from sklearn.linear_model import LogisticRegression


In [0]:
# Initialize the TFIDF vectorizer, that uses bigrams (2,2).
vectorizer = TfidfVectorizer(ngram_range=(2, 2), min_df=4, use_idf=True, smooth_idf=True)



In [0]:
text_clf = Pipeline([('tfidf',vectorizer),('clf',RandomForestClassifier())])

In [32]:
text_clf.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=4, ngram_range=(2, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                               

In [0]:
predictions = text_clf.predict(X_test)

In [0]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score


In [35]:
print(confusion_matrix(y_test,predictions))

[[ 411   54   97   22   16   24   41    0   25   58  222]
 [  53  126   22    0    0    0    6    0    1    2   28]
 [  87   25  248    2    3    8   83    1   54   17  131]
 [  18    0    1  100    0    2    0    0    0   10   14]
 [  11    1    7    0  101    0   14    0    1    2   23]
 [  49    0   14    0    2  186   44    0    0   19   18]
 [  13    2   67    1    8   30 1201    1    2    0   40]
 [   0    0    2    0    0    0    0    2    1    0    0]
 [  15    1   56    0    0    1    2    0  108    3   19]
 [  66    3   40    8    1   13    6    0    7  134   67]
 [ 222   23  133    8   16   19   70    0   21   56  306]]


In [36]:
print(classification_report(y_test,predictions))

                                              precision    recall  f1-score   support

                           Clinical Research       0.43      0.42      0.43       970
              Data Management and Statistics       0.54      0.53      0.53       238
                  Manufacturing & Operations       0.36      0.38      0.37       659
  Medical Affairs / Pharmaceutical Physician       0.71      0.69      0.70       145
   Medical Information and Pharmacovigilance       0.69      0.63      0.66       160
                    Pharmaceutical Marketing       0.66      0.56      0.60       332
Pharmaceutical, Healthcare and Medical Sales       0.82      0.88      0.85      1365
                                    Pharmacy       0.50      0.40      0.44         5
                           Quality-assurance       0.49      0.53      0.51       205
                          Regulatory Affairs       0.45      0.39      0.41       345
                                     Science       0.

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss

In [0]:
text_clf2 = Pipeline([('tfidf',vectorizer),('clf',LogisticRegression())])

In [39]:
text_clf2.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=4, ngram_range=(2, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling

In [0]:
predictions2 = text_clf2.predict(X_test)

In [41]:
print(confusion_matrix(y_test,predictions2))

[[ 481   39   77   19   15   28   62    0   17   52  180]
 [  73  101   27    0    0    0    2    0    1    0   34]
 [  96   25  224    0    0    8  127    0   38   15  126]
 [  23    0    0   89    0    0    1    0    0    9   23]
 [  13    1    5    0   75    0   28    0    1    6   31]
 [  38    0    7    0    0  162   85    0    0   26   14]
 [  20    2   18    1    2   16 1279    0    0    4   23]
 [   0    0    4    0    0    0    1    0    0    0    0]
 [  28    2   64    0    0    0    6    0   80    0   25]
 [ 103    3   43    8    0    9    7    0    8   97   67]
 [ 268   20  126    5    7   13  105    0   14   35  281]]


In [42]:
print(classification_report(y_test,predictions2))

                                              precision    recall  f1-score   support

                           Clinical Research       0.42      0.50      0.46       970
              Data Management and Statistics       0.52      0.42      0.47       238
                  Manufacturing & Operations       0.38      0.34      0.36       659
  Medical Affairs / Pharmaceutical Physician       0.73      0.61      0.67       145
   Medical Information and Pharmacovigilance       0.76      0.47      0.58       160
                    Pharmaceutical Marketing       0.69      0.49      0.57       332
Pharmaceutical, Healthcare and Medical Sales       0.75      0.94      0.83      1365
                                    Pharmacy       0.00      0.00      0.00         5
                           Quality-assurance       0.50      0.39      0.44       205
                          Regulatory Affairs       0.40      0.28      0.33       345
                                     Science       0.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
text_clf3 = Pipeline([('tfidf',vectorizer),('clf',SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=6, tol=None))])

In [44]:
text_clf3.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=4, ngram_range=(2, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', lo

In [0]:
predictions3 = text_clf3.predict(X_test)

In [46]:
print(confusion_matrix(y_test,predictions3))

[[ 422   60   57   30   23   30   90    0   30   64  164]
 [  44  142   20    0    0    0    6    0    3    0   23]
 [  82   25  175    6    3   12  174    0   57   24  101]
 [   9    0    0  109    4    0    2    0    0    5   16]
 [   5    1    2    1  105    0   26    0    1    3   16]
 [  23    1    3    0    2  164  101    0    0   26   12]
 [  10    3    8    4    6   16 1310    0    0    2    6]
 [   0    0    0    0    0    0    5    0    0    0    0]
 [  17    4   48    0    0    0   14    0  110    2   10]
 [  73    4   41   10    2   10   18    0    9  127   51]
 [ 223   25  110   14   20   17  158    0   27   52  228]]


In [47]:
print(classification_report(y_test,predictions3))

                                              precision    recall  f1-score   support

                           Clinical Research       0.46      0.44      0.45       970
              Data Management and Statistics       0.54      0.60      0.56       238
                  Manufacturing & Operations       0.38      0.27      0.31       659
  Medical Affairs / Pharmaceutical Physician       0.63      0.75      0.68       145
   Medical Information and Pharmacovigilance       0.64      0.66      0.65       160
                    Pharmaceutical Marketing       0.66      0.49      0.56       332
Pharmaceutical, Healthcare and Medical Sales       0.69      0.96      0.80      1365
                                    Pharmacy       0.00      0.00      0.00         5
                           Quality-assurance       0.46      0.54      0.50       205
                          Regulatory Affairs       0.42      0.37      0.39       345
                                     Science       0.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
